# High-level CNN Keras (TF) Example

*Modified by Jordan A Caraballo Vega (jordancaraballo)*

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"
MULTI_GPU = True

import warnings # make notebook more readable and nice
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import keras as K
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from keras.utils import multi_gpu_model
from common.params import *
from common.utils import *

Using TensorFlow backend.


In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [3]:
# Performance Improvement
# 1. Make sure channels-first (not last)
K.backend.set_image_data_format('channels_first')

In [4]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tensorflow.__version__)
print(K.backend.backend())
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.6.7 | packaged by conda-forge | (default, Jul  2 2019, 02:18:42) 
[GCC 7.3.0]
Keras:  2.2.4
Numpy:  1.17.0
Tensorflow:  1.13.1
tensorflow
channels_first
GPU:  ['Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB']
CUDA Version 10.1.243
CuDNN Version  No CUDNN in this machine


In [5]:
CPU_COUNT = multiprocessing.cpu_count()
GPU_COUNT = len(get_gpu_name())
print("CPUs: ", CPU_COUNT)
print("GPUs: ", GPU_COUNT)

CPUs:  48
GPUs:  4


In [6]:
def create_symbol(n_classes=N_CLASSES):
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu',
                     input_shape=(3, 32, 32)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    return model

In [7]:
def init_model(m, lr=LR, momentum=MOMENTUM):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(lr, momentum),
        metrics = ['accuracy'])
    return m

In [8]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...


/home/jacaraba/.conda/envs/ngaml/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:441: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. The passed value of 'all' is the default and can simply be removed.
  DeprecationWarning)


(50000, 3, 32, 32) (10000, 3, 32, 32) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 583 ms, sys: 663 ms, total: 1.25 s
Wall time: 3.8 s


In [9]:
%%time
# Load symbol
if MULTI_GPU:
    with tensorflow.device('/cpu:0'):
        sym = create_symbol()
        model_sym = multi_gpu_model(sym, gpus=GPU_COUNT)
else:
    model_sym = create_symbol()

CPU times: user 788 ms, sys: 917 ms, total: 1.71 s
Wall time: 3.63 s


In [10]:
%%time
# Initialise model
model = init_model(model_sym)

CPU times: user 24.3 ms, sys: 1.92 ms, total: 26.2 ms
Wall time: 24.9 ms


In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_1_input (InputLayer)     (None, 3, 32, 32)    0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 3, 32, 32)    0           conv2d_1_input[0][0]             
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 3, 32, 32)    0           conv2d_1_input[0][0]             
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 3, 32, 32)    0           conv2d_1_input[0][0]             
__________________________________________________________________________________________________
lambda_4 (

In [12]:
%%time
# Main training loop: 1m16s
EPOCHS=40
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/40
50000/50000 [==============================] - 57s 1ms/step - loss: 2.0699 - acc: 0.2384
Epoch 2/40
50000/50000 [==============================] - 4s 77us/step - loss: 1.7268 - acc: 0.3780
Epoch 3/40
50000/50000 [==============================] - 4s 77us/step - loss: 1.5221 - acc: 0.4476
Epoch 4/40
50000/50000 [==============================] - 4s 77us/step - loss: 1.3904 - acc: 0.4987
Epoch 5/40
50000/50000 [==============================] - 4s 77us/step - loss: 1.2884 - acc: 0.5351
Epoch 6/40
50000/50000 [==============================] - 4s 77us/step - loss: 1.2026 - acc: 0.5700
Epoch 7/40
50000/50000 [==============================] - 4s 77us/step - loss: 1.1246 - acc: 0.5981
Epoch 8/40
50000/50000 [==============================] - 4s 77us/step - loss: 1.0528 - acc: 0.6224
Epoch 9/40
50000/50000 [==============================] - 4s 77us/step - loss: 0.9906 - acc: 0.6496
Epoch 10/40
50000/50000 [==============================] - 4s 77us/step - loss: 0.9333 - acc: 0.6685

In [13]:
%%time
# Main evaluation loop
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 1.62 s, sys: 235 ms, total: 1.86 s
Wall time: 647 ms


In [14]:
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.799
